In [1]:
#!pip install --user google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 13.3 MB/s eta 0:00:0000:01


In [7]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
import os

os.environ['GOOGLE_API_KEY']="AIzaSyBGPZtYGI-ncmc-xxxxxxxxxxxxxx"

In [9]:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [10]:
#List the available models
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [5]:
#set the model to gemini-pro

model = genai.GenerativeModel('gemini-pro')

##### Features supported by gemini-pro:

- Generate text responses from text inputs.
- Use Gemini for multi-turn conversations (chat).
- Use embeddings for large language models.

### Generate text responses from text inputs.

In [6]:
%%time
response = model.generate_content("How old is the universe? & Why is the only planet with life is Earth?")

CPU times: user 12.6 ms, sys: 607 µs, total: 13.2 ms
Wall time: 7.57 s
CPU times: user 13.2 ms, sys: 689 µs, total: 13.9 ms
Wall time: 7.57 s


In [14]:
to_markdown(response.text)

> **Age of the Universe:**
> 
> The age of the universe is estimated to be around **13.8 billion years**. This estimate is based on various scientific observations, including measurements of the cosmic microwave background radiation, the expansion rate of the universe, and the ages of the oldest stars.
> 
> **Why Earth is the Only Planet with Life (as far as we know):**
> 
> There is still no definitive answer to why Earth is the only known planet with life. However, scientists have identified several factors that may have contributed to the development of life here:
> 
> * **Liquid water:** Liquid water is essential for life as we know it. Earth's proximity to the Sun creates the right temperature conditions for liquid water to exist on its surface.
> * **Oxygen-rich atmosphere:** Oxygen is another key element for life. Earth's atmosphere has a high concentration of oxygen, which is produced by photosynthetic organisms.
> * **Organic molecules:** The building blocks of life, such as amino acids and nucleic acids, may have formed on Earth in ancient oceans or through processes like hydrothermal vents.
> * **Stable environment:** Earth's climate has remained relatively stable for billions of years, providing a conducive environment for life to evolve and thrive.
> * **Plate tectonics:** Plate tectonics has played a role in creating diverse habitats and nutrient cycling, both of which are important for life.
> * **Lack of Jupiter-like planets:** Other planets in our solar system, such as Jupiter, have strong gravitational fields that could disrupt the formation of life on Earth.
> 
> It's important to note that the presence of life on Earth is still a subject of ongoing research and debate. Scientists continue to search for signs of life elsewhere in the universe, but so far, Earth remains the only known planet with confirmed life.

In [15]:
#If the API failed to return a result, use GenerateContentResponse.prompt_feedback to see if it was blocked due to safety concerns regarding the prompt.
response.prompt_feedback

### Chat conversations

Gemini enables you to have freeform conversations across multiple turns. The ChatSession class simplifies the process by managing the state of the conversation, so unlike with generate_content, you do not have to store the conversation history as a list.

In [27]:
#Initialize Chat

model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])
chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-pro',
        generation_config={},
        safety_settings={},
        tools=None,
        system_instruction=None,
    ),
    history=[]
)

In [28]:
response = chat.send_message("In one sentence, explain how does a quantum computer work to a young child.")
to_markdown(response.text)

> Quantum computers use special bits called qubits that can be in multiple states at the same time, allowing them to solve problems much faster than regular computers.

In [29]:
chat.history

[parts {
   text: "In one sentence, explain how does a quantum computer work to a young child."
 }
 role: "user",
 parts {
   text: "Quantum computers use special bits called qubits that can be in multiple states at the same time, allowing them to solve problems much faster than regular computers."
 }
 role: "model"]

In [30]:
for message in chat.history:
    display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))

> **user**: In one sentence, explain how does a quantum computer work to a young child.

> **model**: Quantum computers use special bits called qubits that can be in multiple states at the same time, allowing them to solve problems much faster than regular computers.

In [31]:
model.count_tokens("In one sentence, explain how does a quantum computer work to a young child.")

total_tokens: 16

### Use embeddings

Embedding is a technique used to represent information as a list of floating point numbers in an array. With Gemini, you can represent text (words, sentences, and blocks of text) in a vectorized form, making it easier to compare and contrast embeddings. For example, two texts that share a similar subject matter or sentiment should have similar embeddings, which can be identified through mathematical comparison techniques such as cosine similarity. For more on how and why you should use embeddings, refer to the Embeddings guide(https://ai.google.dev/docs/embeddings_guide).

Use the embed_content method to generate embeddings. The method handles embedding for the following tasks (task_type):

    Task Type	Description
    
    RETRIEVAL_QUERY	Specifies the given text is a query in a search/retrieval setting.
    RETRIEVAL_DOCUMENT	Specifies the given text is a document in a search/retrieval setting. Using this task type requires a title.
    SEMANTIC_SIMILARITY	Specifies the given text will be used for Semantic Textual Similarity (STS).
    CLASSIFICATION	Specifies that the embeddings will be used for classification.
    CLUSTERING	Specifies that the embeddings will be used for clustering.

In [33]:
result = genai.embed_content(
    model="models/embedding-001",
    content="What is the meaning of being human?",
    task_type="retrieval_document",
    title="Embedding of single string")

# 1 input > 1 vector output
print(str(result['embedding'])[:50], '... TRIMMED]')

[-0.004564704, -0.020940237, -0.014654078, 0.00754 ... TRIMMED]


In [34]:
result = genai.embed_content(
    model="models/embedding-001",
    content= ["What is the meaning of being human?", "How does a human showcase humanity?"],
    task_type="retrieval_document",
    title="Embedding of single string")

# A list of inputs > A list of vectors output
for v in result['embedding']:
  print(str(v)[:50], '... TRIMMED ...')

[-0.004564704, -0.020940237, -0.014654078, 0.00754 ... TRIMMED ...
[0.019193657, -0.010098224, 0.0055754078, 0.002361 ... TRIMMED ...


### Multi-turn Chat

While the genai.ChatSession class shown earlier can handle many use cases, it does make some assumptions. If your use case doesn't fit into this chat implementation it's good to remember that genai.ChatSession is just a wrapper around GenerativeModel.generate_content. In addition to single requests, it can handle multi-turn conversations.

The individual messages are glm.Content objects or compatible dictionaries, as seen in previous sections. As a dictionary, the message requires role and parts keys. The role in a conversation can either be the user, which provides the prompts, or model, which provides the responses.

Pass a list of **glm.Content** objects and it will be treated as multi-turn chat:

In [36]:
model = genai.GenerativeModel('gemini-pro')

messages = [
    {'role':'user',
     'parts': ["Briefly explain how a computer works to a young child."]}
]
response = model.generate_content(messages)

to_markdown(response.text)

> Imagine a super smart robot helper called a computer. It's like a wizard that can do lots of things when we tell it what to do.
> 
> Inside the computer, there's a tiny brain called a processor, which is like the boss. It has special parts called memory for remembering things, like your favorite games or movies.
> 
> The computer has wires like roads that connect everything. It uses electricity to power up and make everything work.
> 
> When you click on something with the mouse or type on the keyboard, it sends messages to the brain. The brain reads the messages and decides what to do.
> 
> For example, if you click on a game icon, the brain loads the game from memory so you can play it. It makes the characters move and the music play.
> 
> Just like you need to know words to talk, computers need programs to tell them what to do. Programs are like recipes that the brain follows.
> 
> And that's how a computer works! It's a magical tool that helps us learn, play, and create amazing things.

Note: For multi-turn conversations, you need to send the whole conversation history with each request. The API is stateless.

In [37]:


messages.append({'role':'model',
                 'parts':[response.text]})

messages.append({'role':'user',
                 'parts':["Okay, how about a more detailed explanation to a high school student?"]})

response = model.generate_content(messages)

to_markdown(response.text)

> **How a Computer Works for a High School Student**
> 
> A computer is an electronic device that can be programmed to carry out a set of instructions. It consists of hardware and software.
> 
> **Hardware** is the physical components of a computer, such as the processor, memory, storage devices, input devices (e.g., keyboard, mouse), and output devices (e.g., monitor, printer).
> 
> **Software** is a set of instructions that tells the computer what to do. Software includes operating systems, application programs, and drivers.
> 
> **How a Computer Works**
> 
> 1. **Input:** The user enters data and instructions into the computer using input devices such as the keyboard or mouse.
> 2. **Processing:** The processor, or CPU (Central Processing Unit), executes the instructions and performs calculations.
> 3. **Memory:** The computer stores data and instructions in memory, which is a temporary storage area.
> 4. **Storage:** The computer stores data and instructions permanently on storage devices such as hard drives or solid-state drives.
> 5. **Output:** The computer displays the results of the processing on output devices such as the monitor or printer.
> 
> **Key Components of a Computer**
> 
> * **Processor (CPU):** The brain of the computer that executes instructions and performs calculations.
> * **Memory (RAM):** Temporary storage area that holds data and instructions being processed by the CPU.
> * **Storage (Hard Drive/SSD):** Permanent storage area that stores data and instructions.
> * **Graphics Card:** Processes and displays images on the monitor.
> * **Network Card:** Connects the computer to a network.
> * **Operating System:** Software that manages the computer's hardware and software resources.
> * **Application Programs:** Software that performs specific tasks, such as word processing, web browsing, or playing games.
> 
> **How a Computer Program Works**
> 
> A computer program is a set of instructions that tells the computer what to do. Programs are written in programming languages, which are translated into machine code that the computer can understand.
> 
> When a program is executed, the computer loads the program into memory and the CPU begins executing the instructions one by one. The CPU reads data from memory or storage, performs calculations, and stores the results back in memory or storage.
> 
> **Example**
> 
> Let's say you want to use a word processor to write a letter.
> 
> 1. **Input:** You type the text of the letter using the keyboard.
> 2. **Processing:** The word processor software processes the text, checking for spelling and grammar errors, and formatting the text.
> 3. **Memory:** The word processor stores the text of the letter in memory.
> 4. **Storage:** When you save the letter, the word processor stores the file on your hard drive.
> 5. **Output:** You can print the letter using a printer or display it on the monitor.

The generation_config argument allows you to modify the generation parameters. Every prompt you send to the model includes parameter values that control how the model generates responses.

In [52]:
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content(
    'what is fire in molecular physics',
    generation_config=genai.types.GenerationConfig(
        # Only one candidate for now.
        candidate_count=1,
        stop_sequences=['x'],
        max_output_tokens=20,
        temperature=1.0)
)

In [53]:
text = response.text

if response.candidates[0].finish_reason.name == "MAX_TOKENS":
    text += '?'

to_markdown(text)

> In molecular physics, fire refers to a chemical process characterized by a rapid o